<h3>Pure face detection notebook</h3>

In [2]:
import os
import cv2
import tkinter as tk
from tkinter import simpledialog, messagebox
from deepface import DeepFace


<h3>Files managment</h3>

In [3]:

USER_DB_DIR = 'user_db'
if not os.path.exists(USER_DB_DIR):
    os.makedirs(USER_DB_DIR)


<h3>Method to hold Login,Register</h3>

In [4]:
def capture_image(username):
    cap = cv2.VideoCapture(0)
    while True:
        _, frame = cap.read()
        cv2.imshow('Press "s" to Save, "q" to Quit', frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            image_path = os.path.join(USER_DB_DIR, f'{username}.jpg')
            cv2.imwrite(image_path, frame)
            messagebox.showinfo("Registration", f"User '{username}' registered successfully!")
            break
        elif key & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def register_user():
    username = simpledialog.askstring("Input", "Enter your name:")
    if not username:
        messagebox.showerror("Error", "No name entered.")
        return
    capture_image(username)

def login_user():
    cap = cv2.VideoCapture(0)
    while True:
        _, frame = cap.read()
        cv2.imshow('Press "s" to Login, "q" to Quit', frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            temp_image_path = os.path.join(USER_DB_DIR, 'temp.jpg')
            cv2.imwrite(temp_image_path, frame)
            break
        elif key & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            return
    cap.release()
    cv2.destroyAllWindows()
    
    for filename in os.listdir(USER_DB_DIR):
        if filename == 'temp.jpg':
            continue
        registered_image_path = os.path.join(USER_DB_DIR, filename)
        try:
            result = DeepFace.verify(temp_image_path, registered_image_path, model_name='VGG-Face', enforce_detection=False)
            if result['verified']:
                messagebox.showinfo("Login", f"Welcome, {filename.split('.')[0]}!")
                os.remove(temp_image_path)
                return
        except Exception as e:
            print(f"Error comparing images: {e}")
    
    messagebox.showerror("Login Failed", "No matching user found.")
    os.remove(temp_image_path)




<h3>Main window GUI and controller</h3>

In [5]:
def main():
    root = tk.Tk()
    root.title("Face Recognition System")
    root.geometry("300x200")

    register_button = tk.Button(root, text="Register", command=register_user, width=20, height=2)
    register_button.pack(pady=20)

    login_button = tk.Button(root, text="Login", command=login_user, width=20, height=2)
    login_button.pack(pady=20)
    root.mainloop()

if __name__ == "__main__":
    main()